### Boiler Plate Code

In [1]:
import random
import os
from time import time

import numpy as np
from tqdm import tqdm
from QAP_heuristic import QAP_heuristic

### Simmulated Annealing 

In [2]:
class SimmulatedAnnealing(QAP_heuristic):
    def __init__(self, w, d, rate, Tmin, T0) -> None:
        super().__init__(w, d)
        self.rate = rate
        self.Tmin = Tmin
        self.T0   = T0

    def solve(self, n_iter):
        
        # initialise a random solution 
        X = np.array(list(range(self.n)))
        np.random.shuffle(X)
        
        # code for simmulated annealing
        perm = X
        curr_cost = self.cost(perm)
        best_cost = curr_cost
        T = self.T0
        
        best_cost_history = [best_cost]

        # time

        while T > self.Tmin:
            
            if time() > self.MAX_CPU_TIME: break

            for _ in range(n_iter):
                
                if time() > self.MAX_CPU_TIME: break
                
                new_perm = self.gen_neighbour(perm)
                new_cost = self.cost(new_perm)
                
                ΔE = new_cost - curr_cost
                if (ΔE <= 0) or (np.exp( -ΔE/T ) >= np.random.uniform()):
                    perm = new_perm
                    curr_cost = new_cost
                    
                    if curr_cost < best_cost:
                        best_cost = curr_cost
                        best_perm = perm
                        
    
                best_cost_history.append(best_cost)
        
            T *= self.rate
            
        return best_perm, best_cost, best_cost_history
    
    @staticmethod
    def gen_neighbour(perm):
    # randomly generates a neighbor through swapping two indices
    # with probability 10%, we shuffle the entire array and return

        new_perm = perm[:]
        # Two random indexes
        i = random.randint(0, len(perm)-1)
        j = random.randint(0, len(perm)-1)

        new_perm[i], new_perm[j] = new_perm[j], new_perm[i]   

        if random.random() < 0.1:
            np.random.shuffle(new_perm)

        return new_perm

### Automated Testing

Tests simmulated annealing on QAPLib instances

In [9]:
import csv

instance_path = '../QAPInstances/'
soln_path     = '../QAPSolns/'

# for managing file opening and closing
def read_integers(filename):
    with open(filename) as f:
        return [int(elem) for elem in f.read().split()]


def open_solution(filename: str):
    file_it = iter(read_integers(filename))
    _ = next(file_it)    # this is just how the files within the lib are formatted
    return next(file_it)

        

def test_hueristic(n_iters=10_000, tai_only=False):
    """
    n_iters: number of iterations on each test instance
    tai_only: if True, restricts testing to the Tai instances, reducing computation time
    """
    
    results = [] # for storing results

    with open('../results/SimmulatedAnnealing.csv', mode='w') as f:
        writer = csv.writer(f)
        for filename in os.listdir(instance_path):
            
            # skips any instances that are not Tai
            if tai_only and 'tai' not in filename: continue  

            file_it = iter(read_integers(instance_path+filename))

            # open QAP instance param's 
            n = next(file_it)
            w = np.array([[next(file_it) for j in range(n)] for i in range(n)])
            d = np.array([[next(file_it) for j in range(n)] for i in range(n)])

            # generate an instance
            heuristic = SimmulatedAnnealing(w, d, rate=0.98,Tmin=1_000,T0=100_000_000)

            # open up corresponding soln from QAPLib: 
            soln_file = filename[:-4]+'.sln' # this removes the .dat from filename
            
            try:
                qap_soln = open_solution(soln_path+soln_file)
                _, huerstic_cost, _ = heuristic.solve(n_iters)

                # compute gap
                gap = 100*(huerstic_cost - qap_soln)/qap_soln
                results.append((soln_file, gap))

                print(filename, gap)
                writer.writerow([filename, gap])
                

            # any instances without corresponding solution files are deleted
            except FileNotFoundError:
                os.remove(instance_path+filename)

    return

In [10]:
test_hueristic(n_iters=10_000, tai_only=True)

tai35b.dat 26.735067691067812
tai15a.dat 6.057483758957688
tai35a.dat 12.746232249188894
tai15b.dat 0.6273395512991452
tai80b.dat 34.24289550845903
tai150b.dat 24.515819201453315
tai80a.dat 12.352776286329604
tai100a.dat 11.621374902113605
tai25a.dat 11.725448402064329
tai100b.dat 32.6019835088215
tai25b.dat 23.91106518985317
tai64c.dat 5.928570504890276
tai12a.dat 7.143875659489519
tai12b.dat 2.9755612103659135
tai50b.dat 35.19672421988876
tai50a.dat 13.468707757923186
tai30b.dat 19.99972177799594
tai30a.dat 11.553637606660851
tai40a.dat 13.18570286395041
tai17a.dat 8.515855652159768
tai256c.dat 10.706428926247138
tai60b.dat 37.086511837636955
tai40b.dat 34.17023147370822
tai60a.dat 13.380614552227724
tai20a.dat 10.02897017976295
tai20b.dat 8.513980515619743
[('tai35b.sln', 26.735067691067812), ('tai15a.sln', 6.057483758957688), ('tai35a.sln', 12.746232249188894), ('tai15b.sln', 0.6273395512991452), ('tai80b.sln', 34.24289550845903), ('tai150b.sln', 24.515819201453315), ('tai80a.sln',